# 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [1]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
import requests
# request로 가져온 웹사이트의 html 태그를 찾기위해 사용
from bs4 import BeautifulSoup as bs
# 간격을 두고 가져오기 위해 사용
import time

In [2]:
# 120 다산 콜센터의 첫 페이지를 먼저 불러와 크롤링할 내용을 봅니다.
base_url = "https://opengov.seoul.go.kr/civilappeal/list?items_per_page=1000&"

In [3]:
# pd.read_htm 을 통해 해당 URL의 table 정보를 읽어옵니다.
table = pd.read_html(base_url, encoding="utf8")[0]
table

,번호,제목,생산일,조회수
0,2645,아빠 육아휴직 장려금,2021-08-17,144
1,2644,[서울산업진흥원] 서울메이드란?,2021-06-29,166
2,2643,(강북구) 정비중,2021-06-02,215
3,2642,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,288
4,2641,마포 뇌병변장애인 비전센터,2021-03-12,441
...,...,...,...,...
995,1650,2 이상의 주택을 가지고 있는 경우 아파트 분양대상은 몇 개인가요?,2009-10-16,358
996,1649,전몰군경유족회와 전몰군경미망인회 대표자 및 전화번호는?,2009-10-16,282
997,1648,무허가건물을 가지고 있습니다. 분양대상자에 해당 되는지요?,2009-10-16,434
998,1647,상가소유자가 아파트를 배정받을 수 있습니까?,2009-10-16,494


## 상세정보를 위한 링크정보 수집
* get
* post

In [4]:
# 웹페이지의 결과를 받아옵니다.
response = requests.get( base_url )

In [5]:
# 응답코드 확인 200 == OK
response.status_code

200

## BeautifulSoup 을 통해 html 페이지를 읽기 쉽게 만들기

In [6]:
# html 태그를 파싱해 올 수 있도록 합니다.
html = bs(response.text, 'html.parser')

## 상세 정보 수집을 위한 링크 정보 찾기

In [7]:
# BeautifulSoup 의 select 기능을 통한 링크 태그 찾기
# a 태그 안의 상세 페이지 접근을 위한 링크 번호 수집
a_list = html.select("td > a")
a_link_no = [a["href"].replace("/civilappeal/", "") for a in a_list]

In [8]:
table["view_no"] = a_link_no
table

,번호,제목,생산일,조회수,view_no
0,2645,아빠 육아휴직 장려금,2021-08-17,144,23522464
1,2644,[서울산업진흥원] 서울메이드란?,2021-06-29,166,23194045
2,2643,(강북구) 정비중,2021-06-02,215,23032485
3,2642,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,288,22904492
4,2641,마포 뇌병변장애인 비전센터,2021-03-12,441,22477798
...,...,...,...,...,...
995,1650,2 이상의 주택을 가지고 있는 경우 아파트 분양대상은 몇 개인가요?,2009-10-16,358,2898468
996,1649,전몰군경유족회와 전몰군경미망인회 대표자 및 전화번호는?,2009-10-16,282,2896182
997,1648,무허가건물을 가지고 있습니다. 분양대상자에 해당 되는지요?,2009-10-16,434,2896181
998,1647,상가소유자가 아파트를 배정받을 수 있습니까?,2009-10-16,494,2895045


## 특정 페이지를 수집하는 함수 만들기

In [9]:
def get_one_page(page_no):
    """
    120 주요질문의 특정 페이지 목록을 수집
    """
    base_url = f"https://opengov.seoul.go.kr/civilappeal/list?items_per_page=1000&page={page_no}"
    table = pd.read_html(base_url, encoding="utf8")[0]
    # 웹페이지의 결과를 받아옵니다.
    response = requests.get( base_url )
    # html 태그를 파싱해 올 수 있도록 합니다.
    try:
        html = bs(response.text, 'html.parser')
        a_list = html.select("td > a")
        a_link_no = [a["href"].replace("/civilappeal/", "") for a in a_list]
        table["view_no"] = a_link_no
        return table
    except:
        return pd.DataFrame()

In [10]:
# 함수가 잘 동작하는지 확인
get_one_page(1)

,번호,제목,생산일,조회수,view_no
0,2645,아빠 육아휴직 장려금,2021-08-17,144,23522464
1,2644,[서울산업진흥원] 서울메이드란?,2021-06-29,166,23194045
2,2643,(강북구) 정비중,2021-06-02,215,23032485
3,2642,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,288,22904492
4,2641,마포 뇌병변장애인 비전센터,2021-03-12,441,22477798
...,...,...,...,...,...
995,1650,2 이상의 주택을 가지고 있는 경우 아파트 분양대상은 몇 개인가요?,2009-10-16,358,2898468
996,1649,전몰군경유족회와 전몰군경미망인회 대표자 및 전화번호는?,2009-10-16,282,2896182
997,1648,무허가건물을 가지고 있습니다. 분양대상자에 해당 되는지요?,2009-10-16,434,2896181
998,1647,상가소유자가 아파트를 배정받을 수 있습니까?,2009-10-16,494,2895045


In [11]:
# 없는 페이지도 확인
get_one_page(10000)

""


## 반복문을 통한 여러 페이지 수집하기

* 게시물이 없을 때까지 수집하기

In [12]:
# time.sleep을 통해 일정 간격 쉬었다가 가져옵니다.
# 게시물이 없으면 멈춥니다.
page_no = 1
table_list = []
while True:
    table = get_one_page(page_no)
    table_list.append(table)
    page_no = page_no + 1
    time.sleep(0.01)
    if table.empty:
        break

## 데이터 병합하기

In [13]:
# 여러 페이지의 내용을 하나로 병합합니다.
df = pd.concat(table_list)

## 데이터 파일로 저장하기

In [14]:
file_name = "seoul-120-list.csv"

In [15]:
df.to_csv(file_name, index=False)

In [16]:
pd.read_csv(file_name)

,번호,제목,생산일,조회수,view_no
0,2645,아빠 육아휴직 장려금,2021-08-17,144,23522464
1,2644,[서울산업진흥원] 서울메이드란?,2021-06-29,166,23194045
2,2643,(강북구) 정비중,2021-06-02,215,23032485
3,2642,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,288,22904492
4,2641,마포 뇌병변장애인 비전센터,2021-03-12,441,22477798
...,...,...,...,...,...
2640,5,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,2007-01-07,264,2894353
2641,4,수돗물의 톤당 원가는?,2007-01-07,228,19353862
2642,3,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,2007-01-07,203,19699584
2643,2,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,2007-01-07,178,2898293
